# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
%pip install -r requirements.txt
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.3.3-cp312-cp312-win_amd64.whl (11.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import LLMChain
llm_model = "gpt-4o-mini"

llm = ChatOpenAI(temperature=0.9, model=llm_model)

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\ebrady\AppData\Local\Temp\ipykernel_12440\1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [13]:
product = "Fishing Boots"
chain.run(product)

"Choosing a name for a company that specializes in fishing boots can be crucial for branding and market appeal. Here are some ideas that convey purpose, durability, and the spirit of fishing:\n\n1. **Angler's Gear**\n2. **Hook & Sole**\n3. **Catch & Comfort Boots**\n4. **Reel Footwear**\n5. **TideWalker Boots**\n6. **FishFit Footwear**\n7. **LureBoots**\n8. **StreamStep Boots**\n9. **Baited Boots**\n10. **AquaTread**\n\nConsider picking a name that resonates with your target audience and reflects the quality and features of your products. Additionally, check for domain availability if you plan to create a website!"

## SimpleSequentialChain

In [14]:
from langchain_classic.chains import SimpleSequentialChain

In [15]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [16]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [18]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Choosing a name for a company that makes fishing boots can be both fun and strategic. Here are some suggestions that evoke the spirit of fishing while also suggesting durability and comfort:

1. **Angler's Edge**
2. **Catch & Comfort**
3. **ReelStep Boots**
4. **TideRider**
5. **FishFoot Gear**
6. **Hook & Sole**
7. **Aquatic Trek**
8. **WaterWalker Boots**
9. **CastAway Footwear**
10. **FinFootwear**

Make sure to check for domain availability and trademarks when selecting a name!
Introducing durable and comfortable fishing boots, designed for avid anglers. Our brand names evoke adventure, quality, and aquatic enjoyment.

> Finished chain.


'Introducing durable and comfortable fishing boots, designed for avid anglers. Our brand names evoke adventure, quality, and aquatic enjoyment.'

## SequentialChain

In [19]:
from langchain_classic.chains import SequentialChain

In [20]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [21]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [22]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [23]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [24]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [30]:
review = df.Review[2]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
 'English_Review': 'This mattress had a small hole in the top (it took forever to find where it was), and the patches they provided did not work, maybe because it\'s the top of the mattress where it\'s kind of like fabric and a patch won\'t stick. Maybe I got unlucky with a defective mattress, 

## Router Chain

In [26]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [27]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [32]:
from langchain_classic.chains.router import MultiPromptChain
from langchain_classic.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain_core.prompts import PromptTemplate

In [33]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [34]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [35]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [36]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [37]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [38]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\ebrady\AppData\Local\Temp\ipykernel_12440\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [39]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an idealized object known as a "black body." A black body is a perfect absorber of all incident radiation, meaning it absorbs all wavelengths of light and does not reflect any. When heated, a black body emits radiation in a characteristic spectrum that depends solely on its temperature.\n\nThe key features of black body radiation include:\n\n1. **Temperature Dependence**: The intensity and wavelength distribution of the emitted radiation depend on the temperature of the black body. As the temperature increases, the peak wavelength of the emitted radiation shifts to shorter wavelengths (this is described by Wien\'s displacement law).\n\n2. **Planck\'s Law**: The spectral distribution of black body radiation is described by Planck\'s law, which quantifies the intensity of radiation emitted at different wavelengths for a given temperature.\n\n3. **Stefan-Boltzmann Law**: The total energy emitted per unit surface area

In [40]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To solve the problem \\(2 + 2\\), we can break it down into its component parts:\n\n1. Identify the numbers involved: We have the numbers 2 and 2.\n2. Understand the operation: The operation we are performing is addition.\n\nNow, we can add the two numbers together:\n\n\\[\n2 + 2 = 4\n\\]\n\nSo, the answer to the question \\(2 + 2\\) is \\(4\\).'

In [41]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA serves as the genetic blueprint for the organism. Here are several key reasons why this is the case:\n\n1. **Genetic Information**: DNA contains the instructions needed for the development, functioning, growth, and reproduction of all living organisms. It encodes the information necessary to produce proteins, which perform a vast array of functions in the body.\n\n2. **Cellular Function**: Each cell type in the body has specific functions, and the DNA in each cell contains the genes that are necessary for that cell's role. For example, muscle cells have genes that help them contract, while nerve cells have genes that support their ability to transmit signals.\n\n3. **Development and Differentiation**: During the development of an organism, all cells originate from a single fertilized egg, which contains DNA. As cells divide and differentiate into various types, they retain a complete copy of the original DNA, ensuring that they have the 